In [ ]:
!pip install torchsummary

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms,datasets
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm
import os
import numpy as np
import cv2
import zipfile
reprocess_data = True
from PIL import Image
from torchsummary import summary
import time
from torch.utils.tensorboard import SummaryWriter
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_key")
import seaborn as sns


In [ ]:
wandb.login(key=secret_value_0)

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
! unzip ../input/dogs-vs-cats/test1.zip 
! unzip ../input/dogs-vs-cats/train.zip 

In [ ]:
train_path = "./train"
test_path ="./test1"
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

In [ ]:
print(len(train_files))
print(len(test_files))

In [ ]:
imgpath = os.path.join(train_path,train_files[0])
print(imgpath)

In [ ]:
img = cv2.imread(imgpath,0)
type(img)

In [ ]:
imgtensor = torch.tensor(img)

In [ ]:
imgtensor.shape

In [ ]:
# imgtensorresize = transforms.

In [ ]:
class Dataset():
    def __init__(self,filelist,filepath,transform = None):
        self.filelist = filelist
        self.filepath = filepath
        self.transform = transform
    def __len__(self):
        return int(len(self.filelist))
    def __getitem__(self,index):
        imgpath = os.path.join(self.filepath,self.filelist[index])
        img = Image.open(imgpath)
        if "dog" in imgpath:
            label = 1
        else:
            label = 0 
        if self.transform is not None:
            img = self.transform(img)
        return (img,label)
            
        

In [ ]:
transformations = transforms.Compose([transforms.Resize((60,60)),transforms.ToTensor()])
train = Dataset(train_files,train_path,transformations)
test = Dataset(test_files,test_path,transformations)
train_set,val_set = torch.utils.data.random_split(train,[20000,5000]) 

In [ ]:
len(val_set)

In [ ]:
val_set.__getitem__(0)[0].shape

In [ ]:
val_set_bal = [val_set.__getitem__(x)[1] for x in range(len(val_set)) ]

In [ ]:
sns.countplot(val_set_bal)

In [ ]:
print("train set size :",train_set.__len__())
print("val set size :",val_set.__len__())

In [ ]:
train_set.__getitem__(0)[0].shape

In [ ]:
train_dataset = torch.utils.data.DataLoader(dataset = train_set,batch_size = 32,shuffle=True)
test_dataset = torch.utils.data.DataLoader(dataset = val_set,batch_size = 32,shuffle=True)

In [ ]:
# class Covnet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Sequential(
#             nn.Conv2d(3,64,3,padding= 1),
#             nn.ReLU(),
# #             nn.MaxPool2d(2,2),
# #             nn.Dropout(0.1),
#             nn.Conv2d(64,64,3,padding= 1),
#             nn.ReLU(),
#             nn.MaxPool2d((2,2))            
#             ) 
            
#         self.conv2 =   nn.Sequential(
#             nn.Conv2d(64,128,3,padding= 1),
#             nn.ReLU(),
# #             nn.MaxPool2d(2,2),
# #             nn.Dropout(0.1),
#             nn.Conv2d(128,128,3,padding= 1),
#             nn.ReLU(),
#             nn.MaxPool2d((2,2))
#         ) 
#         self.conv3 =   nn.Sequential(
#             nn.Conv2d(128,256,3,padding= 1),
#             nn.ReLU(),
#             nn.Conv2d(256,256,3,padding= 1),
#             nn.ReLU(),
#             nn.Conv2d(256,256,3,padding= 1),
#             nn.ReLU(),
#             nn.MaxPool2d((2,2))
#             )
#         self.conv4 =   nn.Sequential(
#             nn.Conv2d(256,512,3,padding= 1),
#             nn.ReLU(),
#             nn.Conv2d(512,512,3,padding= 1),
#             nn.ReLU(),
#             nn.Conv2d(512,512,3,padding= 1),
#             nn.ReLU(),
#             nn.MaxPool2d((2,2))
#             )
#         self.conv5 =   nn.Sequential(
#             nn.Conv2d(512,512,3,padding= 1),
#             nn.ReLU(),
#             nn.Conv2d(512,512,3,padding= 1),
#             nn.ReLU(),
#             nn.Conv2d(512,512,3,padding= 1),
#             nn.ReLU(),
#             nn.MaxPool2d((2,2))
#             )
#         self.fc1 = nn.Sequential(
#         nn.Flatten(),
#         nn.Linear(512*7*7,4096),
#         nn.ReLU(),
#         nn.Dropout(0.5),
#         nn.Linear(4096,4096),
#         nn.ReLU(),
#         nn.Dropout(0.5),
#         nn.Linear(4096,1000)
#         )
            
#         self.fc2 = nn.Sequential(
#         nn.Linear(1000,2),
#         )
                
#     def forward(self,x):
#         x= self.conv1(x)
#         x= self.conv2(x)
#         x= self.conv3(x)
#         x = self.conv4(x)
#         x = self.conv5(x)
#         x = self.fc1(x)
#         x = self.fc2(x)
#         return F.softmax(x,dim = 1) 
        


In [ ]:
# class Covnet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Sequential(
#             nn.Conv2d(3,10,5),
#             nn.ReLU(),
#             nn.MaxPool2d(2,2)
#             ) 
            
#         self.conv2 =   nn.Sequential(
#             nn.Conv2d(10,16,5),
#             nn.ReLU(),
#             nn.MaxPool2d(2,2)
#         ) 

#         self.fc1 = nn.Sequential(
#         nn.Flatten(),
#         nn.Linear(16*9*9,128),
#         nn.ReLU(),
#         nn.Linear(128,64),
#         nn.ReLU()
#         )
            
#         self.fc2 = nn.Sequential(
#         nn.Linear(64,2),
#         )
                
#     def forward(self,x):
#         x= self.conv1(x)
#         x= self.conv2(x)
#         x = self.fc1(x)
#         x = self.fc2(x)
#         return F.softmax(x,dim = 1) 

In [ ]:
class Covnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
            ) 
            
        self.conv2 =   nn.Sequential(
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        ) 
        self.conv3 =   nn.Sequential(
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        ) 

        self.fc1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*4*4,256),
        nn.ReLU(),
        nn.Linear(256,128),
        nn.ReLU()
        )
            
        self.fc2 = nn.Sequential(
        nn.Linear(128,2),
        )
                
    def forward(self,x):
        x= self.conv1(x)
        x= self.conv2(x)
        x = self.conv3(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return F.softmax(x,dim = 1) 

In [ ]:
model = Covnet()
model.cuda()
summary(model,(3,60,60))

In [ ]:
EPOCHS = 100
optimiser = optim.Adam(model.parameters(),lr=0.0005)
# optimiser = optim.SGD(model.parameters(),lr=1e-5,momentum = 0.9)
LOSS = nn.CrossEntropyLoss().to(device)

In [ ]:
len(train_set)

In [ ]:
len(val_set)

In [ ]:
wandb.init(project="cnn_pytorch_cat&dog",reinit=True)

In [ ]:
writer = SummaryWriter()

In [ ]:
wandb.watch(model,log="all")

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    total = 0
    correct = 0
    train_loss = 0
    train_accuracy = 0
    counter = 0
    train_running_loss =0
    with tqdm(train_dataset, unit="batch") as tepoch:
        tepoch.set_description(f'Epoch {epoch+1}/{EPOCHS}')
        for data,label in tepoch:
            data,label = data.to(device) , label.to(device)
            optimiser.zero_grad()
            output = model(data)
            loss = LOSS(output,label)
            loss.backward()
            optimiser.step() 
            train_running_loss+= loss.item() * data.size(0)
            _,pred = torch.max(output.data,1)
            total += label.size(0)
            correct += (pred == label).sum().item()
        train_accuracy = correct/total
        train_loss = train_running_loss/len(train_set)
        print("=====/train/=====")
        print("epoch {} train accuracy {}".format(epoch+1,train_accuracy))
        print("epoch {} train loss {}".format(epoch+1,train_loss))
        writer.add_scalar("train accuracy",train_accuracy,epoch+1)
        writer.add_scalar("train loss",train_loss,epoch+1)
        wandb.log({"train accuracy":train_accuracy,"train loss":train_loss,"epochs":epoch+1},step = epoch+1)
    if epoch %1 == 0:
        model.eval()
        total = 0
        correct = 0
        val_loss = 0
        val_accuarcy =0
        val_running_loss = 0
        with torch.no_grad():
            for val_data,val_label in test_dataset:
                val_data,val_label = val_data.to(device),val_label.to(device)
                val_output = model(val_data)
                #loss function used is CrossEntropyLoss 
                loss_val = LOSS(val_output,val_label)
                #calacuate the running loss by multiplying loss value by batch size
                val_running_loss+= loss_val.item() * val_data.size(0)
#                 val_running_loss+= loss_val.item() 
                _,pred = torch.max(val_output.data,1)    
                total += val_label.size(0)
                correct += (pred == val_label).sum().item()
            val_accuarcy = correct/total
            #calcuate loss per epoch by dividing runing loss by number of items in validation set
            val_loss = val_running_loss/len(val_set)
            print(val_running_loss)
            print("=====/val/=====")
            print("epoch {} val accuracy {}".format(epoch+1,val_accuarcy))
            print("epoch {} val loss {}".format(epoch+1,val_loss))
            writer.add_scalar("val accuracy",val_accuarcy,epoch+1)
            writer.add_scalar("val loss",val_loss,epoch+1)
            wandb.log({"val accuracy":val_accuarcy,"val loss":val_loss},step = epoch+1)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [ ]:
wandb.finish()